In [15]:
import openai
import os
import asyncio
import os
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAIChatPromptExecutionSettings
from semantic_kernel.functions import KernelArguments
import asyncio
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.prompt_template import PromptTemplateConfig, InputVariable
from semantic_kernel.contents.chat_history import ChatHistory


from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

### Using Semantic Kernel

In [44]:
# Function to get completion using Semantic Kernel
async def get_completion_skernel(prompt, model="gpt-3.5-turbo"):
    kernel = Kernel()

    # Add OpenAI Chat Completion Service
    openai_service = OpenAIChatCompletion(
        api_key=openai.api_key ,
        ai_model_id=model
    )
    kernel.add_service(openai_service)

    # Prepare the chat execution settings
    chat_execution_settings = OpenAIChatPromptExecutionSettings(
        ai_model_id=model,
        max_tokens=1000,
        temperature=0.7,
        top_p=0.9
    )
    chat_completion_service = kernel.get_service(type=ChatCompletionClientBase)  
    chat_history = ChatHistory()

    chat_prompt_template_config = PromptTemplateConfig(
    template=prompt,
    name="grounded_response",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="query_term", description="The user input", is_required=True),
    ],
    execution_settings=chat_execution_settings,
)

    chat_function = kernel.add_function(
        function_name="ChatGPTFunc",
        plugin_name="chatGPTPlugin",
        prompt_template_config=chat_prompt_template_config
    )
    arguments = KernelArguments( query_term=prompt)

    # Invoke the function with the prompt
    result = await kernel.invoke(
                chat_function,arguments
            )

    return result


async def main(prompt):
    result = await get_completion_skernel(prompt)
    print(result)


In [43]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=.4, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

### Few-Shot Prompting

In [45]:
text = f"""
An insurance claim is a formal request made by a policyholder to their insurance company for 
coverage or compensation for a covered loss or policy event. The process involves gathering and processing various details, 
including the policyholder's information, such as policy number, type, coverage details, and demographics.
Claim details include the claim number, date of loss, type of loss, location, description, estimated and actual loss amounts, 
status, and settlement date. Financial information encompasses the premium amount, payment history, claim history, deductible, 
and reimbursement amounts. For auto claims, specific details like the vehicle's VIN, make, model, year, value, and damage details are required. 
In the case of home or property claims, information about the property address, type, value, and damage details is necessary. 
Health claims involve patient information, diagnosis codes, treatment details, provider information, and payment details. 
Fraud indicators are identified through potential red flags, anomalies, and suspicious claim patterns. 
This comprehensive approach is used to evaluate the claim, determine coverage, and decide on the appropriate settlement.
"""

prompt = f"""
Create a detailed description of an insurance claim involving a specific incident type, based on the structured information provided below. 
Ensure that the description covers all relevant aspects of the claim, including policyholder details, claim specifics, financial information, and any other pertinent data. 
The incident type should be clearly defined and integrated into the narrative, illustrating how it impacts the claim process and outcome.
```{text}```
"""

response_claim = await main(prompt) 


Policyholder Details:
- Name: Sarah Johnson
- Policy Number: 123456789
- Coverage: Auto Insurance
- Demographics: 35-year-old female, residing in Los Angeles, CA

Claim Specifics:
- Claim Number: AC987654
- Date of Loss: June 15, 2021
- Type of Loss: Auto Accident
- Location: Intersection of Main St. and Oak Ave.
- Description: Sarah was rear-ended by another driver while stopped at a red light, causing significant damage to her car.
- Estimated Loss Amount: $5,000
- Actual Loss Amount: $4,500
- Status: Under Review
- Settlement Date: Pending

Financial Information:
- Premium Amount: $800 annually
- Payment History: Paid in full for the year
- Claim History: No previous claims
- Deductible: $500
- Reimbursement Amount: $4,000 (after deductible)

Incident Impact on Claim Process:
The auto accident involving Sarah Johnson has prompted her to file a claim with her insurance company for coverage of the damages to her vehicle. The claim process involves verifying the details of the incident

### Zero Shot Prompting

In [34]:
prompt = f"""
Create a detailed description of an insurance claim involving a specific incident type. The description should comprehensively 
cover all aspects of the claim, including policyholder details, claim specifics, financial information, and any other pertinent data. 
Clearly define the incident type and explain how it affects the claim process and outcome. Ensure the narrative 
is complete and informative, highlighting how each piece of information contributes to the overall evaluation and settlement of the claim.
"""
response_claim = await main(prompt) 



Incident Type: Car Accident

Policyholder Details:
- Name: John Smith
- Policy Number: 123456789
- Insurance Company: ABC Insurance Co.
- Coverage: Auto insurance with liability, collision, and comprehensive coverage

Claim Specifics:
- Date of Incident: March 15, 2021
- Location: Intersection of Main St. and Elm St.
- Description: John Smith was driving his car through the intersection when another driver ran a red light and collided with his vehicle. The impact caused significant damage to both cars, and John sustained minor injuries.

Financial Information:
- Cost of Repairs: $5,000 for John's car
- Medical Expenses: $1,000 for John's injuries
- Total Claim Amount: $6,000

Claim Process and Outcome:
- John Smith immediately contacted his insurance company, ABC Insurance Co., to report the accident and file a claim. He provided all necessary details, including the police report, photos of the damage, and medical records.

- An insurance adjuster from ABC Insurance Co. was assigned to

### Structured Prompting

In [35]:
structured_info = {
  "Policyholder Information": {
    "Policy Number": "123456789",
    "Policy Type": "Auto",
    "Coverage Details": {
      "Limits": "$100,000",
      "Deductibles": "$500"
    },
    "Demographics": {
      "Age": 45,
      "Gender": "Male",
      "Location": "New York, NY"
    }
  },
  "Claim Details": {
    "Claim Number": "987654321",
    "Date of Loss": "2024-08-01",
    "Loss Type": "Collision",
    "Loss Location": "Manhattan, NY",
    "Loss Description": "Rear-end collision at a traffic light",
    "Estimated Loss Amount": "$10,000",
    "Actual Loss Amount": "$9,500",
    "Claim Status": "Open",
    "Claim Settlement Date": "Pending"
  },
  "Financial Information": {
    "Premium Amount": "$1,200 annually",
    "Payment History": "Up to date",
    "Claim History": "No previous claims",
    "Deductible Amount": "$500",
    "Reimbursement Amount": "Pending"
  },
  "Specific Incident Information": {
    "Vehicle Information": {
      "VIN": "1HGCM82633A004352",
      "Make": "Honda",
      "Model": "Accord",
      "Year": 2020,
      "Vehicle Value": "$20,000",
      "Damage Details": "Rear bumper and trunk damage"
    }
  },
  "Fraud Indicators": {
    "Red Flags": "None identified",
    "Suspicious Patterns": "None identified"
  },
  "Additional Considerations": {
    "Data Quality": "Verified and consistent",
    "Data Privacy": "Compliant with GDPR",
    "Balance of Data": "Balanced distribution of claim types",
    "Synthetic Data": "No synthetic data used"
  }
}
prompt = f"""
Create a detailed description of an insurance claim involving a collision, based on the structured information provided below. 
Clearly define the incident type and explain how it impacts the claim process and outcome. The narrative 
should be complete and informative, highlighting how each piece of information contributes to the overall evaluation and settlement of the claim.
```{structured_info}```
"""

response_claim = await main(prompt) 

The policyholder, a 45-year-old male residing in New York, NY, holds an auto insurance policy with a coverage limit of $100,000 and a deductible of $500. The policy number is 123456789, with an annual premium of $1,200, and a payment history that is up to date. There are no previous claims on record for this policyholder.

On August 1, 2024, the policyholder was involved in a collision in Manhattan, NY. The incident was a rear-end collision that occurred at a traffic light intersection. The estimated loss amount is $10,000, with the actual loss amount reported as $9,500. The claim status is currently open, with the claim settlement date pending.

The vehicle involved in the collision is a 2020 Honda Accord with a value of $20,000. The VIN is 1HGCM82633A004352, and the vehicle sustained damage to the rear bumper and trunk.

There are no identified fraud indicators or suspicious patterns in this claim. The data quality is verified and consistent, compliant with GDPR regulations, and ther

### Detailed Prompting

In [41]:
prompt = f"""
Policyholder Information:

Start by describing the policyholder's information, including their policy number, policy type, coverage limits, deductibles, and demographics such as age, gender, and location.
Claim Details:

Next, provide details about the claim itself. Include the claim number, the date of loss, the type of loss (e.g., collision, fire, theft), the location where the loss occurred, and a brief description of the incident. Mention both the estimated and actual loss amounts, as well as the current status of the claim (e.g., open, closed, pending) and the settlement date if applicable.
Financial Information:

Outline the financial aspects of the claim. This should include the policyholder's premium amount, their payment history, and any previous claims they’ve made. Specify the deductible amount associated with the policy and any reimbursement amounts that have been processed or are pending.
Specific Incident Information:

If the claim involves a specific asset like a vehicle or property, describe this in detail. For an auto claim, include the vehicle’s VIN, make, model, year, value, and a summary of the damages. For a property claim, provide details such as the property address, type (house, apartment, etc.), value, and extent of the damage.
Fraud Indicators:

Assess the claim for any potential red flags or anomalies that might indicate fraud. Mention any patterns of suspicious claims or behaviors that were observed during the evaluation.
Additional Considerations:

Finally, discuss any additional considerations that were taken into account during the claim processing. This might include the quality and consistency of the data, adherence to data privacy and security regulations, and whether the data was balanced or if synthetic data was used.
Conclusion:

Summarize how the incident type impacts the claim process and outcome. Ensure the narrative is cohesive and informative, highlighting how each piece of information contributes to the overall evaluation and settlement of the claim and do write it as a story

"""
response_claim = await main(prompt) 


Policyholder Information:
Policyholder: John Smith
Policy Number: 123456789
Policy Type: Auto Insurance
Coverage Limits: $50,000
Deductible: $500
Demographics: Male, 35 years old, located in Los Angeles, CA

Claim Details:
Claim Number: 987654321
Date of Loss: June 15, 2021
Type of Loss: Collision
Location: Intersection of Main St. and Elm St., Los Angeles, CA
Description: John was involved in a collision with another vehicle at the intersection. Estimated loss amount is $10,000, actual loss amount is $9,500. Claim status is open, pending settlement date.

Financial Information:
Premium Amount: $800 annually
Payment History: Up to date
Previous Claims: No previous claims
Deductible Amount: $500
Reimbursement: $9,500 pending

Specific Incident Information:
Vehicle: 2020 Toyota Camry
VIN: 12345678901234567
Value: $15,000
Damage: Front bumper and fender damage, estimated repair cost of $9,500

Fraud Indicators:
No red flags or anomalies indicating fraud were observed during the evaluation

## Testing of Synthetic Data in different Use Cases 

### Policy Summarization

In [37]:

prompt = f"""
Based on the detailed insurance claim description provided below, create a concise summary that highlights the most important aspects of the claim. 
The summary should include the incident type, key claim details (such as dates, amounts, and status), relevant policyholder information, 
and the final outcome or resolution of the claim. The goal is to provide a clear and quick overview of the claim in a few sentences
```{response_claim}```
"""
response_summary = await main(prompt) 


Summary: The insurance claim involved a car accident that occurred on May 15, 2021, resulting in $5,000 worth of damages. The policyholder, John Smith, filed the claim promptly and provided all necessary documentation. The claim was approved by the insurance company, and John received full reimbursement for the repairs.


### Claim Processing

In [38]:
prompt = f"""
Based on the detailed insurance claim description provided below, outline the steps involved in processing this claim. 
Include key actions such as verifying the policyholder's information, assessing the incident and damages, 
calculating the estimated and actual loss amounts, checking the policy coverage and deductibles, determining the claim status, and finalizing the claim settlement. 
The goal is to provide a clear and comprehensive sequence of actions necessary to process this claim effectively.
```{response_claim}```
"""
response_processing = await main(prompt) 


1. Verify the policyholder's information: Gather the policyholder's name, contact information, policy number, and any other relevant details to confirm their identity and coverage.

2. Assess the incident and damages: Review the detailed description of the incident provided by the policyholder, including the date, time, location, and circumstances surrounding the claim. Evaluate the extent of the damages or loss incurred.

3. Calculate the estimated loss amount: Based on the information provided, estimate the cost of repairs, replacements, or other expenses related to the claim. Consider obtaining quotes or assessments from professionals if necessary.

4. Check the policy coverage and deductibles: Review the policy terms and coverage limits to determine the extent to which the claim is covered. Take note of any deductibles or limits that may apply to the claim.

5. Determine the actual loss amount: Once the damages have been assessed and the policy coverage confirmed, calculate the fin

### Policy Checking

In [42]:
prompt = f"""
Using the detailed insurance claim description provided below, perform a policy check to ensure that the incident type is covered under the policy. 
Evaluate whether the policyholder meets all the necessary conditions, such as coverage limits, deductibles, and policy validity. 
The result should confirm if the policy supports the claim, and identify any potential issues or gaps in coverage that could affect the claim's outcome.
```{response_claim}```
"""
response_check = await main(prompt) 


I'm sorry, but I cannot perform a policy check as I do not have access to specific insurance policy information. It is recommended to contact the insurance company directly or review the policy documents to determine if the incident type described in the insurance claim is covered under the policy. Make sure to check for coverage limits, deductibles, and policy validity to ensure that the claim is eligible for reimbursement. If you have any specific questions or need assistance interpreting the policy terms, feel free to provide more details, and I'll do my best to help.
